In [1]:
import nltk
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.linear_model import LogisticRegression
import re

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [2]:
stopwords = stopwords.words('english')[:]

In [4]:
pd.set_option("display.max_rows",270)

# Task A

### Engagement Score

In [21]:
post = pd.read_csv("insta_data_with_labels.csv").iloc[:,1:]

In [23]:
max_like = max(post["likes"])
max_like

1539774

In [24]:
max_like = max(post["likes"])
list_norm_like = []
for p in post.itertuples():
    norm_like = p.likes/max_like
    list_norm_like.append(norm_like)
post["norm_like"] = list_norm_like

max_comment = max(post["comments"])
list_norm_comment = []
for p in post.itertuples():
    norm_comment = p.comments/max_comment
    list_norm_comment.append(norm_comment)
    
post["norm_comment"]= list_norm_comment

In [25]:
engagement_score = []
for p in post.itertuples():
    e_score = .4*p.norm_like+.6*p.norm_comment
    engagement_score.append(e_score)
engagement_score

[0.017560655627206684,
 0.02366236449064101,
 0.037809338825375745,
 0.09019406690899491,
 0.11252898280760024,
 0.12070288926685988,
 0.0684502432182017,
 0.07439259702505385,
 0.029396026017428026,
 0.044150234902194656,
 0.08526724827039955,
 0.06025193407719019,
 0.12698927056147763,
 0.15885322135378185,
 0.06036176590398917,
 0.062475533746834275,
 0.07939817644979734,
 0.10823153463623596,
 0.06637011537807715,
 0.0524855773452004,
 0.09030157519960061,
 0.08801654248682615,
 0.07918353096459323,
 0.10909306582838439,
 0.33221843192878076,
 0.08145705246775684,
 0.04558561823412439,
 0.12271770638568008,
 0.058085794017089146,
 0.0878135163021513,
 0.032037474945001886,
 0.03397843335917866,
 0.03537797494481368,
 0.05954897098807081,
 0.26121185644432376,
 0.24268638545740612,
 0.14122382089357513,
 0.1482207042829119,
 0.09477811178034269,
 0.043401602872195966,
 0.1697252296637716,
 0.05805284878448901,
 0.02543701615277202,
 0.040006916819220216,
 0.05253045494082848,
 0.097

In [26]:
med_e_score = np.median(engagement_score)

In [27]:
eng_score = []
for i in range(len(engagement_score)):
    if engagement_score[i]>=med_e_score:
        e_score = 1
    else:
        e_score = 0
    eng_score.append(e_score)
eng_score
post["engagement_score"] = eng_score

In [28]:
post

,Unnamed: 0.1,urls,captions,comments,likes,labels,norm_like,norm_comment,engagement_score
0,0,https://scontent-sea1-1.cdninstagram.com/vp/7831adc9c2d624937ec8197b4affcded/5C76DA0B/t51.2885-15/e15/c181.0.718.718/s480x480/44904753_194498844802328_8238377254064471543_n.jpg?ig_cache_key=MTkxMjcyNTE4Njk1Njg2MzM4Mg%3D%3D.2.c,"Photo by @amivitale | Ntipiyon Nonguta and her son Bernard, one, relax with their neighbors as they make beaded belts for theLoisaba Community Trust in Ewaso Village in Laikipia, Kenya. These women are in a community surrounding Loisaba Conservancy (@loisaba_conservancy) and are being supported and given access to international markets. Revenue from ecotourism is reinvested into neighboring communities like #Ewaso . To get involved and learn more about community based conservation, please follow @loisaba_conservancy and @amivitale. @nature_org @natgeoimagecollection #kenya #magicalkenya #whyilovekenya #northernkenya #africa #children #mothersandsons #mothers #empowerment #conservation #everydayafrica #photojournalism #amivitale",278,61459,"['people', 'public space', 'community', 'tribe', 'tree', 'travel', 'temple', 'recreation', 'tourism', 'crowd']",0.039914,0.002658,0
1,1,https://scontent-sea1-1.cdninstagram.com/vp/53f7539662906933bbbce050cd4fdc2c/5BEF2CD7/t51.2885-15/e15/s640x640/45543088_970491189813847_3800824331312248168_n.jpg,"Video by @amivitale | Wild adult elephants drink and cool off at a watering on the Namunyak Wildlife Conservancy. An elephant’s trunk typically holds around half of a gallon of water, but a big bull elephant can hold as much as 2.5 gallons. After spraying themselves, the elephants will often coat themselves with dust which acts as a bug repellent and sunblock. Namunyak Wildlife Conservancy is home to Reteti Elephant Sanctuary (@r.e.s.c.u.e), the first ever community-owned and run elephant sanctuary in Africa. Reteti operates in partnership with Conservation International (@conservationorg) who provide critical operational support and work to scale the Reteti community-centered model to create lasting impacts worldwide. Follow @amivitale @conservationorg & @r.e.s.c.u.e to learn about the incredible work being done to protect and preserve these magical beings. @nrt_kenya @kenyawildlifeservice @sandiegozoo @tusk_org @thephotosociety @natgeo #namunyak #protectelephants #bekindtoelephants #DontLetThemDisappear #elephants #saveelephants #stoppoaching #kenya #northernkenya #magicalkenya #whyilovekenya #africa #everydayafrica #natureisspeaking #photojournalism #amivitale",362,83092,NaN,0.053964,0.003461,0
2,2,https://scontent-sea1-1.cdninstagram.com/vp/1e4ff18aa09db5069b6e1c6e9e65eac0/5C6F3508/t51.2885-15/e15/s480x480/44577672_336304220507048_1738524933735107944_n.jpg?ig_cache_key=MTkxMjQyNTAwNDk3MzMzNTE0NQ%3D%3D.2,"Photo by George Steinmetz @geosteinmetz | Epic flying last week over the Pyramids of Giza. I had tried for years, and was turned down even with Egyptian military helicopters. Last week Egypt hosted the world paramotor championships, and suddenly the impossible happened. This is me, the last one to land at sunset. For a rare view of the pyramids from above, go to @geosteinmetz",431,136026,"['air sports', 'paragliding', 'landscape', 'sand', 'ecoregion', 'desert', 'powered paragliding', 'windsports', 'sahara', 'steppe']",0.088342,0.004121,0
3,3,https://scontent-sea1-1.cdninstagram.com/vp/a43b38e630c9b7b76c93d76517e4882e/5C8A9361/t51.2885-15/e15/s480x480/42697363_1856252364473290_1258265168604898264_n.jpg?ig_cache_key=MTkxMjM1MDIwNDMwMDk5NzU5Mg%3D%3D.2,"Photo by @irablockphoto | A nighttime view from the top of the Old Town Bridge Tower of the Charles Bridge and Prague Castle along the banks of the Vltava River in Prague, Czech Republic. The Charles Bridge was constructed under King Charles IV of Bohemia starting in 1357 and was finished in the begging of the 15th century. The bridge is a very popular tourist destination especially at sunset and early evening when the city lights up

# Task B

## Logistic regression

### Labels

In [30]:
post.dropna(axis=0, how='any', inplace=True)

In [31]:
label=post["labels"].values.tolist()
label=" ".join(label)
label = label.lower()        

In [32]:
# tokenize the text 
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
labels = tokenizer.tokenize(label)

In [33]:
filter_labels = Counter(x for x in labels if x not in stopwords and x.isdigit() == False and len(x) != 1 and x.isalpha() == True)

In [34]:
filter_labels.most_common()[:100]

[('sky', 78),
 ('wildlife', 48),
 ('tree', 46),
 ('mammal', 45),
 ('water', 44),
 ('fauna', 39),
 ('mountain', 30),
 ('photography', 30),
 ('grass', 30),
 ('ecosystem', 29),
 ('wilderness', 28),
 ('animal', 28),
 ('organism', 27),
 ('rock', 24),
 ('terrestrial', 24),
 ('recreation', 22),
 ('nature', 22),
 ('girl', 22),
 ('plant', 21),
 ('elephant', 21),
 ('landscape', 20),
 ('bird', 19),
 ('darkness', 18),
 ('snout', 18),
 ('sea', 17),
 ('fun', 17),
 ('temple', 16),
 ('phenomenon', 16),
 ('cloud', 16),
 ('reflection', 15),
 ('landforms', 15),
 ('atmosphere', 15),
 ('human', 15),
 ('forest', 15),
 ('geological', 14),
 ('highland', 14),
 ('park', 14),
 ('black', 14),
 ('night', 14),
 ('grassland', 14),
 ('mountainous', 13),
 ('marine', 13),
 ('national', 13),
 ('formation', 13),
 ('city', 12),
 ('reserve', 12),
 ('like', 12),
 ('site', 12),
 ('hill', 12),
 ('ice', 12),
 ('ecoregion', 11),
 ('monochrome', 11),
 ('horizon', 11),
 ('morning', 11),
 ('area', 11),
 ('tourism', 10),
 ('eye', 1

In [35]:
useless_words=[]
for i in range(100,len(filter_labels.most_common())):
    useless_words.append(filter_labels.most_common()[i][0])


In [152]:
natural_objects=["autumn","spring","reflections","phenomenon","wood","nature","natrual",'penguins','cattle','species','insect','flower', 'whiskers','amphibian','fauna','elephant','animal','whale','bird','reptile','fish','grass','tree','plant','bird','cats','dog','horse','lions','bear','mammoths','elephants','wildlife','mammal',"tiger","snake"]
human = ["immigrants","population","dancer","male","female",'women','refugee',"snout",'chest','eye','hair','face','child','human',"family","children",'girl',"girls","people",'photojournalism',"community","communityies",'migrants',"man"]
geological  = ["desert","national park",'formation', 'geological','terrain','badlands',"landscape",'mountains',"mountain"'wilderness','forest','highland','grassland','hill','steppe','jungle',"vallay",'horizon','ecosystem','rock','terrestrial','landforms','landform','ecoregion',"geology","hillside"]
aerial =['cloud','sky','sunrise', 'star','atmosphere','aerial','light','darkness', 'sunset','night','astronomical' ,"sun","sunlight"]
activity = ["swimming","marriage","workshop",'festival','cycling','music','fun','climbing','religion','carnival','flame','ceremony','wedding', 'ritual','travel',"paragliding",'game', "recreation",'event','tradition','safari','vacation','sports','sport','competition event', 'boating',"boat"]
artificial_objects = ["pyramids","conservation","artificial",'painting','photography','sanctuary','aircraft',"food",'house','metropolis','outerwear', 'wall','design','room','arts','village','hall','fashion accessory','history', 'furniture','infrastructure', 'campfire','window','building','temple','city','street','vehicle','architecture','leisure','tourism','town','urban','computer','structure',"castle","cities"'town','landmark','town square', 'plaza', 'tourist attraction', 'road',"robots","transportation" ]
water=['island',"islands",'ocean','sea','iceberg', 'arctic ocean', 'shore','sea',"wave",'ice', 'shore','arctic','glacial ', 'polar ice cap', 'ice', 'melting', 'sea', 'ice cap','river','underwater',"coast","snow"]

In [106]:
def replacement(text):
    
    text = text.lower()
    
    for w in useless_words:
        try:
            text = text.replace(w," ")
        except:
            pass   
    for w in human:
        try:
            text = text.replace(w," human ")
        except:
            pass   
    for w in natural_objects:
        try:
            text = text.replace(w, ' natural ')
        except:
            pass
    for w in geological:
        try:
            text = text.replace(w, ' geological ')
        except:
            pass
    for w in aerial:
        try:
            text = text.replace(w, ' aerial ')
        except:
            pass
    for w in activity:
        try:
            text = text.replace(w, ' activity ')
        except:
            pass
    for w in water:
        try:
            text = text.replace(w, ' water ')
        except:
            pass

    for w in artificial_objects:
        try:
            text = text.replace(w, ' artificial ')
        except:
            pass
  

    return text

In [38]:
label_column = []
for p in post['labels']:
    new_text =replacement(p)
    #a= tokenizer.tokenize(new_text)
    label_column.append(new_text)

In [39]:
len(post['labels'])

238

In [40]:
len(post["engagement_score"])

238

In [65]:
def cal_dfitf_score(text,min_df):
    vectorizer=CountVectorizer(stop_words="english", analyzer='word', min_df=min_df, max_features=None)
    transformer=TfidfTransformer()
    tfidf=transformer.fit_transform(vectorizer.fit_transform(text))
    word=vectorizer.get_feature_names()
    weight=tfidf.toarray()
    a=[]
    for i in range(len(weight)):
        print (u"-------post",i,u"tf-idf------")
        a.append(weight[i])
        for j in range(len(word)):
            print (word[j],weight[i][j])
    return a,word



In [66]:
a,word=cal_dfitf_score(label_column,0.1)

-------post 0 tf-idf------
activity 0.42611673082867557
aerial 0.0
artificial 0.8749325806248831
geological 0.0
hu 0.0
human 0.0
ism 0.0
natural 0.23003806439144883
water 0.0
-------post 1 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
geological 1.0
hu 0.0
human 0.0
ism 0.0
natural 0.0
water 0.0
-------post 2 tf-idf------
activity 0.0
aerial 0.22435681553919523
artificial 0.959873379365778
geological 0.0
hu 0.0
human 0.0
ism 0.0
natural 0.16824718394681354
water 0.0
-------post 3 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
geological 0.7920315462330343
hu 0.0
human 0.0
ism 0.0
natural 0.6104801632909205
water 0.0
-------post 4 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
geological 0.0
hu 0.0
human 0.0
ism 0.621331823231409
natural 0.5793197900616783
water 0.527574967452803
-------post 5 tf-idf------
activity 0.0
aerial 0.4570866145734026
artificial 0.0
geological 0.8894221870280872
hu 0.0
human 0.0
ism 0.0
natural 0.0
water 0.0
-------post 6 tf-idf------
activity 

human 0.0
ism 0.0
natural 1.0
water 0.0
-------post 74 tf-idf------
activity 0.0
aerial 0.9572366235578617
artificial 0.0
geological 0.0
hu 0.25116977928234413
human 0.0
ism 0.0
natural 0.14356806223810067
water 0.0
-------post 75 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
geological 0.0
hu 0.0
human 0.0
ism 0.0
natural 1.0
water 0.0
-------post 76 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
geological 0.1954530468284964
hu 0.2635609517388986
human 0.2743894023420296
ism 0.0
natural 0.9039049657382835
water 0.0
-------post 77 tf-idf------
activity 0.0
aerial 0.0
artificial 0.6863466002764965
geological 0.0
hu 0.6314051841814494
human 0.0
ism 0.0
natural 0.36090973619131544
water 0.0
-------post 78 tf-idf------
activity 0.0
aerial 0.3802488424644604
artificial 0.0
geological 0.9248842185941103
hu 0.0
human 0.0
ism 0.0
natural 0.0
water 0.0
-------post 79 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
geological 0.231956683832599
hu 0.0
human 0.0
ism 0.3835051471084

natural 0.9230435377095128
water 0.0
-------post 169 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
geological 0.5442188074011354
hu 0.0
human 0.0
ism 0.0
natural 0.8389433173170199
water 0.0
-------post 170 tf-idf------
activity 0.0
aerial 0.7787256975895518
artificial 0.5552747641057446
geological 0.0
hu 0.0
human 0.0
ism 0.0
natural 0.291986685074809
water 0.0
-------post 171 tf-idf------
activity 0.0
aerial 0.3400335194286793
artificial 0.0
geological 0.0
hu 0.0
human 0.0
ism 0.5469725052514827
natural 0.764982538469907
water 0.0
-------post 172 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
geological 0.2595830656952966
hu 0.3500378272945837
human 0.36441919637478676
ism 0.42918117362027264
natural 0.6002424262780439
water 0.36441919637478676
-------post 173 tf-idf------
activity 0.0
aerial 0.5741257068032487
artificial 0.8187671664079296
geological 0.0
hu 0.0
human 0.0
ism 0.0
natural 0.0
water 0.0
-------post 174 tf-idf------
activity 0.8259353694162292
aerial 0.297288

activity 0.0
aerial 0.0
artificial 0.0
geological 0.0
hu 0.2790821967476758
human 0.29054834058179474
ism 0.34218251685092865
natural 0.7976136760301858
water 0.29054834058179474
-------post 224 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
geological 0.0
hu 0.6440788248679273
human 0.6705409228896186
ism 0.0
natural 0.3681539597601638
water 0.0
-------post 225 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
geological 0.0
hu 0.0
human 0.0
ism 0.0
natural 0.0
water 0.0
-------post 226 tf-idf------
activity 0.215095715560352
aerial 0.0
artificial 0.0
geological 0.3013032364675622
hu 0.0
human 0.0
ism 0.0
natural 0.9289511251092559
water 0.0
-------post 227 tf-idf------
activity 0.0
aerial 0.324114607104463
artificial 0.0
geological 0.946017823014725
hu 0.0
human 0.0
ism 0.0
natural 0.0
water 0.0
-------post 228 tf-idf------
activity 0.0
aerial 0.8142068933038921
artificial 0.580574831435556
geological 0.0
hu 0.0
human 0.0
ism 0.0
natural 0.0
water 0.0
-------post 229 tf-idf---

In [67]:
df=pd.DataFrame(a)
df.columns=word

In [74]:
df

,activity,aerial,artificial,geological,hu,human,ism,natural,water
0,0.426117,0.000000,0.874933,0.000000,0.000000,0.000000,0.000000,0.230038,0.000000
1,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.224357,0.959873,0.000000,0.000000,0.000000,0.000000,0.168247,0.000000
3,0.000000,0.000000,0.000000,0.792032,0.000000,0.000000,0.000000,0.610480,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.621332,0.579320,0.527575
5,0.000000,0.457087,0.000000,0.889422,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.885091,0.000000,0.000000,0.000000,0.000000,0.465418,0.000000
7,0.270895,0.195013,0.834330,0.000000,0.000000,0.000000,0.000000,0.438725,0.000000
8,0.000000,0.000000,0.000000,0.460623,0.000000,0.000000,0.000000,0.887596,0.000000
9,0.000000,0.766281,0.000000,0.372767,0.000000,0.000000,0.000000,0.000000,0.523314


In [76]:
df.drop(['hu',"ism"],axis=1,inplace=True)

In [77]:
y=post["engagement_score"]

In [78]:
y

0      0
2      0
3      1
4      1
5      1
6      0
7      0
9      0
10     0
11     0
12     1
13     1
15     0
16     0
17     1
18     0
19     0
20     1
21     1
22     0
23     1
24     1
25     0
26     0
27     1
28     0
29     0
30     0
31     0
32     0
33     0
34     1
35     1
36     1
37     1
38     1
39     0
40     1
41     0
44     0
45     1
46     0
47     1
48     1
49     0
50     0
52     1
53     1
54     1
55     0
56     0
57     0
59     0
61     0
62     1
63     1
64     0
65     0
66     1
67     1
68     0
69     1
71     1
72     1
73     1
74     1
75     1
76     1
77     0
78     1
79     1
80     0
81     1
82     0
83     0
84     1
85     1
86     0
87     0
88     1
89     0
91     0
92     1
93     0
94     1
95     1
96     0
97     0
98     0
100    0
102    1
103    1
104    1
106    1
107    1
108    1
109    0
110    0
111    0
112    0
113    0
114    0
115    1
117    1
118    1
119    1
120    1
121    1
124    0
125    0
126    1
1

In [79]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
def logictic_pred(df,y):
    X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=0)
    lr = LogisticRegression() 
    lr.fit(X_train, y_train)
    y_pred=lr.predict(X_test)
    print("confusion_matrix:\n",confusion_matrix(y_test, y_pred))
    print("accuracy_score:\n",accuracy_score(y_test, y_pred))

In [199]:
logictic_pred(df,y)

confusion_matrix:
 [[12 11]
 [ 3 22]]
accuracy_score:
 0.7083333333333334


### Captions

In [69]:
text=post["captions"].values.tolist()
text=" ".join(text)
text = text.lower()
tokens = tokenizer.tokenize(text)

In [70]:
filter_words = Counter(x for x in tokens if x not in stopwords and x.isdigit() == False and len(x) != 1 and x.isalpha() == True)

In [71]:
filter_words.most_common()[:300]

[('photo', 260),
 ('one', 89),
 ('see', 80),
 ('people', 80),
 ('follow', 77),
 ('years', 74),
 ('world', 73),
 ('natgeo', 65),
 ('many', 57),
 ('year', 56),
 ('like', 54),
 ('first', 53),
 ('wildlife', 50),
 ('time', 49),
 ('work', 47),
 ('life', 46),
 ('national', 45),
 ('new', 44),
 ('species', 41),
 ('part', 41),
 ('old', 40),
 ('around', 40),
 ('park', 40),
 ('back', 39),
 ('story', 38),
 ('china', 36),
 ('way', 35),
 ('still', 34),
 ('long', 34),
 ('wild', 34),
 ('women', 33),
 ('get', 32),
 ('conservation', 32),
 ('land', 32),
 ('made', 32),
 ('animals', 31),
 ('last', 29),
 ('day', 29),
 ('make', 28),
 ('also', 28),
 ('found', 28),
 ('kenya', 27),
 ('africa', 27),
 ('today', 27),
 ('images', 27),
 ('nature', 27),
 ('across', 27),
 ('find', 27),
 ('city', 25),
 ('war', 25),
 ('poaching', 25),
 ('live', 25),
 ('project', 25),
 ('assignment', 25),
 ('since', 25),
 ('children', 24),
 ('home', 24),
 ('elephants', 24),
 ('red', 24),
 ('water', 23),
 ('visit', 23),
 ('along', 22),
 ('

In [ ]:
post[post['captions'].str.contains('photojournalism')]

In [114]:
if "artificial" in useless_words:
    print("y")

y


In [116]:
def clean_data(text):
    word=[]
    #delete punctuations stopwords
    words = re.compile(r'[^a-zA-Z@.]+').split(text)
    for w in words:
        if w not in stopwords and len(w)>2:
            word.append(w)
    return word

In [194]:
a=[]
z=[]
captions_column=[]
for p in post['captions']:
    a=clean_data(p)
    for i in a:
        i=i.lower()
        if i in human:
            z.append("human")
        if i in natural_objects:
            z.append("natural")
        if i in artificial_objects:
            z.append("artificial")
        if i in activity:
            z.append("activity")
        if i in water:
            z.append("water")
        if i in aerial:
            z.append("aerial")
        if i in geological:  
            z.append("geologicial")
    n= ",".join(z)
    captions_column.append(n)

In [196]:
b,w=cal_dfitf_score(captions_column,0.1)

-------post 0 tf-idf------
activity 0.0
aerial 0.0
artificial 0.4472135954999579
geologicial 0.0
human 0.8944271909999159
natural 0.0
water 0.0
-------post 1 tf-idf------
activity 0.0
aerial 0.0
artificial 0.6401843996644799
geologicial 0.0
human 0.7682212795973759
natural 0.0
water 0.0
-------post 2 tf-idf------
activity 0.0
aerial 0.08583050245515372
artificial 0.8511524409399777
geologicial 0.0
human 0.5106914645639866
natural 0.0
water 0.08583050245515372
-------post 3 tf-idf------
activity 0.0
aerial 0.08489748558173238
artificial 0.8419000240656187
geologicial 0.0
human 0.5051400144393713
natural 0.0
water 0.16979497116346476
-------post 4 tf-idf------
activity 0.0
aerial 0.14690578863023684
artificial 0.7284078329039916
geologicial 0.0
human 0.5827262663231932
natural 0.1481403855664696
water 0.2938115772604737
-------post 5 tf-idf------
activity 0.0
aerial 0.12145435669925433
artificial 0.6022111557681893
geologicial 0.1229886733866117
human 0.5419900401913703
natural 0.1224750

artificial 0.36067710595104113
geologicial 0.06641524076491143
human 0.6267503808329568
natural 0.583215883509977
water 0.3100461842546597
-------post 90 tf-idf------
activity 0.16850199525036585
aerial 0.08939892882196059
artificial 0.36052599583023787
geologicial 0.07242263485185993
human 0.6264877960328723
natural 0.5829715380243404
water 0.3099162865827967
-------post 91 tf-idf------
activity 0.16723083039703407
aerial 0.09463947822521294
artificial 0.35780621809745256
geologicial 0.07187628459723786
human 0.6217616248906553
natural 0.5905030034711152
water 0.30757830423194205
-------post 92 tf-idf------
activity 0.16366574944619866
aerial 0.09262192320729747
artificial 0.37888153603959807
geologicial 0.07034400270623684
human 0.6257285973987302
natural 0.5779144693827064
water 0.30102125042371675
-------post 93 tf-idf------
activity 0.15937080416743968
aerial 0.09019132246685647
artificial 0.374528843106247
geologicial 0.0742061916480131
human 0.6260780660880547
natural 0.58548605

activity 0.1212166019696894
aerial 0.06883080274748496
artificial 0.34128604688636627
geologicial 0.10138230268689084
human 0.6329304869528974
natural 0.630993246694564
water 0.23465046391188055
-------post 154 tf-idf------
activity 0.12025361285420826
aerial 0.07138780315022496
artificial 0.34165269766983136
geologicial 0.1005768845147637
human 0.6279022551769874
natural 0.6353701111413013
water 0.2358901321485694
-------post 155 tf-idf------
activity 0.12312623196309751
aerial 0.07116961243109896
artificial 0.3406084653989623
geologicial 0.10026947986310984
human 0.6290516703314168
natural 0.6334281566988967
water 0.23826348509541825
-------post 156 tf-idf------
activity 0.12282833258490886
aerial 0.07099742017806626
artificial 0.345906616781197
geologicial 0.1031527216023849
human 0.6275297030101361
natural 0.6318956006302245
water 0.23768701537874357
-------post 157 tf-idf------
activity 0.12839086969728497
aerial 0.07050205204725851
artificial 0.3556521820712201
geologicial 0.1024

-------post 207 tf-idf------
activity 0.13258262423746908
aerial 0.08287550629583437
artificial 0.3401540578863953
geologicial 0.11422779483589615
human 0.6026890690067674
natural 0.6616116068103558
water 0.21409505793090547
-------post 208 tf-idf------
activity 0.1317283765984243
aerial 0.08234152828781836
artificial 0.3379623996466209
geologicial 0.11349180982569539
human 0.6010740664856009
natural 0.6642682150200366
water 0.2150028794181924
-------post 209 tf-idf------
activity 0.13159763347920547
aerial 0.08225980262985773
artificial 0.3376269650238807
geologicial 0.11337916687354214
human 0.6004774881297207
natural 0.6636089151738509
water 0.21935947367962058
-------post 210 tf-idf------
activity 0.1357402651558974
aerial 0.0818721340593263
artificial 0.33603581893892565
geologicial 0.11284484101638337
human 0.6044134192995441
natural 0.6604815028617287
water 0.2183256908248701
-------post 211 tf-idf------
activity 0.1355347689681812
aerial 0.08174818843854784
artificial 0.3355270

In [197]:
df1=pd.DataFrame(b)
df1.columns=w

In [198]:
logictic_pred(df1,y)

confusion_matrix:
 [[ 0 23]
 [ 0 25]]
accuracy_score:
 0.5208333333333334
